In [ ]:
import os

base_path = "/content/ECB_project"

folders = [
    "data/raw_html",
    "data/raw_text",
    "data/processed",
    "src",
    "logs"
]

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)


In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [ ]:
# scrape_ecb_urls.py
# Robust (no selenium here):
#   1) requests main archive page
#   2) extract yearly snippet URLs from data-snippets
#   3) requests each snippet page and collect statement links
#   4) fetch each statement page -> meeting_date + title
# Output: ecb_urls_dates.csv with columns [meeting_date, url, title]
# ------------------------------------------------------------

import re
import time
from urllib.parse import urljoin, urlparse

import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

BASE = "https://www.ecb.europa.eu"
INDEX_URL = "https://www.ecb.europa.eu/press/press_conference/monetary-policy-statement/html/index.en.html"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def parse_date_from_is_url(url: str) -> str | None:
    """
    Supports:
      .../html/is000203.en.html
      .../html/is030508_1.en.html
      .../html/ecb.is251218~hash.en.html
    """
    m = re.search(r"(?:/|\.)(?:ecb\.)?is(\d{2})(\d{2})(\d{2})(?:[_~].*)?\.en\.html$", url)
    if not m:
        return None
    yy, mm, dd = map(int, m.groups())
    year = 1900 + yy if yy >= 90 else 2000 + yy
    return f"{year:04d}-{mm:02d}-{dd:02d}"

def is_ecb_domain(url: str) -> bool:
    p = urlparse(url)
    return (p.netloc == "" or p.netloc.endswith("ecb.europa.eu"))

def fetch_soup(url: str) -> BeautifulSoup:
    r = requests.get(url, headers=HEADERS, timeout=40)
    r.raise_for_status()
    return BeautifulSoup(r.content, "html.parser")

def extract_title(soup: BeautifulSoup) -> str | None:
    """
    Prefer og:title (usually clean), then <title>, then first <h1>.
    """
    og = soup.find("meta", {"property": "og:title"})
    if og and og.get("content"):
        return og["content"].strip()

    t = soup.find("title")
    if t and t.get_text(strip=True):
        return t.get_text(strip=True)

    h1 = soup.find("h1")
    if h1 and h1.get_text(strip=True):
        return h1.get_text(strip=True)

    return None

def url_priority(u: str) -> int:
    """
    Lower score = preferred.
    Heuristics to pick the 'canonical' page if multiple URLs share the same date.
    """
    score = 0
    if "~" in u:            # hash urls often duplicates
        score += 10
    if re.search(r"_\d+\.en\.html$", u):  # isYYMMDD_1.en.html etc.
        score += 5
    if "ecb.is" in u:       # newer hashed patterns
        score += 3
    return score

def main():
    print("Fetching index page…", flush=True)
    index_soup = fetch_soup(INDEX_URL)

    dl = index_soup.find("dl", id="lazyload-container")
    if dl is None or not dl.has_attr("data-snippets"):
        raise RuntimeError("Could not find data-snippets on the index page.")

    snippets_raw = dl["data-snippets"]
    snippet_paths = [s.strip() for s in snippets_raw.split(",") if s.strip()]
    snippet_urls = [urljoin(INDEX_URL, p) for p in snippet_paths]

    # keep only 1999+
    snippet_urls_1999 = []
    for u in snippet_urls:
        m = re.search(r"/(\d{4})/html/index_include\.en\.html$", u)
        if m and int(m.group(1)) >= 1999:
            snippet_urls_1999.append(u)

    print(f"Keeping {len(snippet_urls_1999)} snippet pages for 1999+.", flush=True)

    all_statement_urls = set()

    for u in tqdm(snippet_urls_1999, desc="Downloading snippet pages"):
        try:
            soup = fetch_soup(u)

            for a in soup.find_all("a", href=True):
                href = a["href"].strip()
                full = urljoin(BASE, href)

                if not is_ecb_domain(full):
                    continue
                if "/press/press_conference/monetary-policy-statement/" not in full:
                    continue
                if not full.endswith(".en.html"):
                    continue

                all_statement_urls.add(full)

        except Exception as e:
            print(f"\nFailed snippet: {u} | {e}", flush=True)

        time.sleep(0.05)

    all_statement_urls = sorted(all_statement_urls)
    print(f"\nCollected {len(all_statement_urls)} statement URLs (1999+ snippets).", flush=True)

    df = pd.DataFrame({"url": all_statement_urls})

    meeting_dates = []
    titles = []

    for url in tqdm(df["url"], desc="Extracting dates + titles"):
        date = None
        title = None

        try:
            page = fetch_soup(url)

            # date: meta published_time if available
            meta = page.find("meta", {"property": "article:published_time"})
            if meta and meta.get("content"):
                dt = pd.to_datetime(meta["content"], errors="coerce")
                if pd.notna(dt):
                    date = dt.strftime("%Y-%m-%d")

            # title
            title = extract_title(page)

        except Exception:
            pass

        if not date:
            date = parse_date_from_is_url(url)

        meeting_dates.append(date)
        titles.append(title)

        time.sleep(0.05)

    df["meeting_date"] = meeting_dates
    df["title"] = titles

    # keep 1999+
    out = df.dropna(subset=["meeting_date"]).copy()
    out = out[out["meeting_date"] >= "1999-01-01"].copy()

    # choose ONE row per date
    out["prio"] = out["url"].apply(url_priority)
    out = (
        out.sort_values(["meeting_date", "prio", "url"])
           .drop_duplicates(subset=["meeting_date"], keep="first")
           .drop(columns=["prio"])
           .sort_values("meeting_date")
           .reset_index(drop=True)
    )

    out_path = "ecb_urls_dates.csv"
    out[["meeting_date", "url", "title"]].to_csv(out_path, index=False)

    print("\nSaved:", out_path, flush=True)
    print("Rows:", len(out), flush=True)
    if len(out):
        print("Earliest:", out["meeting_date"].iloc[0], flush=True)
        print("Latest:", out["meeting_date"].iloc[-1], flush=True)

if __name__ == "__main__":
    main()


Fetching index page…
Keeping 27 snippet pages for 1999+.



Collected 285 statement URLs (1999+ snippets).



Extracting dates + titles: 100%|██████████| 285/285 [03:39<00:00,  1.30it/s]


Saved: ecb_urls_dates.csv
Rows: 282
Earliest: 1999-01-07
Latest: 2025-12-18


In [ ]:
import pandas as pd

df = pd.read_csv("ecb_urls_dates.csv")

df["meeting_date"] = df["meeting_date"].astype(str).str.strip()

df["meeting_date"] = pd.to_datetime(
    df["meeting_date"],
    errors="coerce",
    dayfirst=True,
    format="mixed"
)

print("Unparsed rows:", df["meeting_date"].isna().sum())


Unparsed rows: 0


In [ ]:
df["title_lower"] = df["title"].astype(str).str.lower()

keep_pattern = r"(introductory statement|monetary policy statement)"
drop_pattern = r"(transcript|opening|signing|agreement|exhibition|balance of payments|design|assessment)"

df_clean = df[
    df["title_lower"].str.contains(keep_pattern, regex=True, na=False) &
    ~df["title_lower"].str.contains(drop_pattern, regex=True, na=False)
].copy()


/tmp/ipython-input-2378790882.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["title_lower"].str.contains(keep_pattern, regex=True, na=False) &
/tmp/ipython-input-2378790882.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~df["title_lower"].str.contains(drop_pattern, regex=True, na=False)


In [ ]:
df_clean = (
    df_clean.sort_values(["meeting_date", "url"])
            .drop_duplicates(subset=["meeting_date"], keep="first")
            .reset_index(drop=True)
)


In [ ]:
print("Total meetings:", len(df_clean))
print(df_clean.groupby(df_clean["meeting_date"].dt.year).size())


Total meetings: 274
meeting_date
1999    11
2000    13
2001    11
2002    11
2003    12
2004    11
2005    11
2006    12
2007    11
2008    12
2009    12
2010    12
2011    12
2012    12
2013    12
2014    12
2015     8
2016     8
2017     7
2018     8
2019     8
2020     8
2021     8
2022     8
2023     8
2024     8
2025     8
dtype: int64


In [ ]:
df_clean[["meeting_date","url","title"]].to_csv(
    "ecb_monetary_policy_statements_all_years.csv",
    index=False
)


In [ ]:
import re
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup

HEADERS = {"User-Agent": "Mozilla/5.0"}

JUNK_LINE_RE = re.compile(
    r"(?im)^\s*(jump to the transcript of the questions and answers|with the transcript of the questions and answers)\s*$"
)

TRANSCRIPT_START_RE = re.compile(r"(?im)^\s*transcript of the questions.*$")
QNA_HEADING_RE = re.compile(r"(?im)^\s*(questions and answers|q\s*&\s*a|q&a)\s*$")
QUESTION_PREFIX_RE = re.compile(r"(?im)^\s*question\s*:")

SPOKEN_MARKERS = [
    "we are now ready to take your questions",
    "we are now ready for your questions",
    "we are now at your disposal for questions",
    "i will now take your questions",
]

def download_html(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=40)
    r.raise_for_status()
    r.encoding = r.apparent_encoding
    return r.text

def extract_title(soup: BeautifulSoup) -> str:
    h1 = soup.find("h1")
    if h1 and h1.get_text(strip=True):
        return h1.get_text(" ", strip=True)
    if soup.title and soup.title.get_text(strip=True):
        return soup.title.get_text(" ", strip=True)
    return ""

def extract_main_text(soup: BeautifulSoup) -> str:
    main = soup.find("main")
    article = main.find("article") if main else None
    node = article or main or soup.body or soup

    for tag in node.find_all(["script", "style", "nav", "header", "footer", "aside"]):
        tag.decompose()

    text = node.get_text(separator="\n")
    text = re.sub(r"\r", "\n", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

def remove_junk_lines(text: str) -> str:
    lines = text.split("\n")
    lines = [ln for ln in lines if not JUNK_LINE_RE.match(ln.strip())]
    return "\n".join(lines).strip()

def split_statement_qna_whole(text: str):
    text = remove_junk_lines(text)
    lines = text.split("\n")

    def find_line_index(pattern_re):
        for i, ln in enumerate(lines):
            if pattern_re.match(ln.strip()):
                return i
        return None

    i = find_line_index(TRANSCRIPT_START_RE)
    if i is not None:
        return "\n".join(lines[:i]).strip(), "\n".join(lines[i:]).strip()

    i = find_line_index(QNA_HEADING_RE)
    if i is not None:
        return "\n".join(lines[:i]).strip(), "\n".join(lines[i:]).strip()

    i = find_line_index(QUESTION_PREFIX_RE)
    if i is not None:
        return "\n".join(lines[:i]).strip(), "\n".join(lines[i:]).strip()

    low = text.lower()
    for m in SPOKEN_MARKERS:
        idx = low.find(m)
        if idx != -1:
            return text[:idx].strip(), text[idx:].strip()

    return text.strip(), ""

def scrape_one(url: str):
    html = download_html(url)
    soup = BeautifulSoup(html, "html.parser")

    title = extract_title(soup)
    main_text = extract_main_text(soup)
    statement, qa = split_statement_qna_whole(main_text)

    return title, statement, qa

def excel_safe_multiline(s: str) -> str:
    """
    Replace real newlines inside a cell with the literal characters \n
    so Excel doesn't display them as extra rows when opening CSV.
    """
    if s is None:
        return ""
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    return s.replace("\n", r"\n")

def scrape_statements():
    url_index_path = "ecb_monetary_policy_statements_all_years.csv"
    df = pd.read_csv(url_index_path)

    if "meeting_date" not in df.columns or "url" not in df.columns:
        raise ValueError("Your driver CSV must have columns: meeting_date, url")

    out = []
    for _, row in df.iterrows():
        meeting_date = str(row["meeting_date"])[:10]
        url = row["url"]

        try:
            title, statement, qa = scrape_one(url)
        except Exception as e:
            print(f"[FAIL] {meeting_date} {url} -> {e}")
            title = statement = qa = ""

        out.append({
            "meeting_date": meeting_date,
            "url": url,
            "title": title,
            "statement_text": excel_safe_multiline(statement),
            "qa_text": excel_safe_multiline(qa),
        })

    out_df = pd.DataFrame(out)

    # IMPORTANT: quote all fields so Excel behaves better
    out_df.to_csv(
        "ecb_statement_qna_raw.csv",
        index=False,
        quoting=csv.QUOTE_ALL
    )

    print("Saved: ecb_statement_qna_raw.csv")
    print("Rows:", len(out_df))

if __name__ == "__main__":
    scrape_statements()


Saved: ecb_statement_qna_raw.csv
Rows: 274


In [ ]:
import pandas as pd
import re

# load raw CSV
df = pd.read_csv("ecb_statement_qna_raw.csv", dtype=str)

# turn literal \n back into real newlines
for col in ["statement_text", "qa_text"]:
    df[col] = df[col].fillna("").str.replace(r"\n", "\n", regex=False)

# --- STATEMENT CLEANING ---
def clean_statement(s):
    lines = [ln.strip() for ln in s.split("\n") if ln.strip()]

    drop_re = re.compile(
        r"^(ecb|european central bank).*press conference$|"
        r"^monetary policy statement$|"
        r"^introductory statement$",
        re.I
    )

    while lines and drop_re.match(lines[0]):
        lines.pop(0)

    return "\n".join(lines)

# --- Q&A CLEANING ---
def clean_qa(q):
    lines = [ln.strip() for ln in q.split("\n") if ln.strip()]

    lines = [
        ln for ln in lines
        if not re.match(r"^transcript of the questions", ln, re.I)
    ]

    return "\n".join(lines)

df["statement_text_clean"] = df["statement_text"].apply(clean_statement)
df["qa_text_clean"] = df["qa_text"].apply(clean_qa)

# make CSV-safe again
for col in ["statement_text_clean", "qa_text_clean"]:
    df[col] = df[col].str.replace("\n", r"\n", regex=False)

# export to CSV
df.to_csv("ecb_statement_qna_cleaned.csv", index=False)

print("Saved: ecb_statement_qna_cleaned.csv")
print("Rows:", len(df))


Saved: ecb_statement_qna_cleaned.csv
Rows: 274


Tokenisation (texte → mots propres)

In [ ]:
# Step 1.0 — Prepare dates and sort

import pandas as pd

df = df.copy()

# FIX: create date from meeting_date (this column exists)
df["date"] = pd.to_datetime(df["meeting_date"], errors="coerce")

df = df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

print("Date range:", df["date"].min(), "→", df["date"].max())
df[["date"]].head()


In [18]:
# Step 1.1 — Tokenize text (stopwords + Porter stemming)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer

nltk.download("stopwords")

STOP_WORDS = set(stopwords.words("english"))
tokenizer = RegexpTokenizer(r"[a-zA-Z]+")
stemmer = PorterStemmer()

def tokenize(text: str):
    if pd.isna(text):
        return []
    tokens = tokenizer.tokenize(str(text).lower())
    tokens = [stemmer.stem(t) for t in tokens if t not in STOP_WORDS]
    return tokens

print(tokenize(df.loc[0, "statement_text_clean"])[:30])


['ecb', 'press', 'confer', 'introductori', 'statement', 'nwillem', 'f', 'duisenberg', 'presid', 'european', 'central', 'bank', 'thursday', 'januari', 'nladi', 'gentlemen', 'vice', 'presid', 'today', 'report', 'outcom', 'first', 'meet', 'govern', 'council', 'european', 'central', 'bank', 'stage', 'three']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# Step 1.2 — Build bigram sets (paper-faithful, no double processing)

from nltk.util import bigrams

def bigram_set(text: str):
    """
    Convert text -> tokens -> set of bigrams
    Uses tokenize() from Step 1.1 (stopwords + Porter stem) exactly once.
    """
    tokens = tokenize(text)
    if len(tokens) < 2:
        return set()
    return set(bigrams(tokens))

# Quick test
print(list(bigram_set(df.loc[0, "statement_text_clean"]))[:10])


[('trend', 'gener'), ('monetari', 'aggreg'), ('uncertainti', 'origin'), ('trend', 'e'), ('smooth', 'start'), ('novemb', 'latest'), ('object', 'maintain'), ('least', 'anoth'), ('maintain', 'price'), ('nbroadli', 'base')]


In [22]:
# Step 1.3 — Jaccard similarity

def jaccard(A: set, B: set) -> float:
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

# Quick test
A = {("ecb","press"), ("press","conference")}
B = {("ecb","press"), ("central","bank")}
print(jaccard(A, B))  # expected: 1 / 3 = 0.333...


0.3333333333333333


In [24]:
# Step 1.0 — Prepare dates and sort (FIXED)

import pandas as pd

df = df.copy()

df["date"] = pd.to_datetime(df["meeting_date"], errors="coerce")

df = df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

print("Date range:", df["date"].min(), "→", df["date"].max())
df[["meeting_date", "date"]].head()


Date range: 1999-01-07 00:00:00 → 2025-12-18 00:00:00


,meeting_date,date
0,1999-01-07,1999-01-07
1,1999-02-04,1999-02-04
2,1999-03-04,1999-03-04
3,1999-04-08,1999-04-08
4,1999-05-06,1999-05-06


In [25]:
# Step 1.4 — Statement similarity over time (consecutive meetings)

import pandas as pd

df = df.sort_values("date").reset_index(drop=True)

statement_sets = df["statement_text_clean"].apply(bigram_set)

statement_similarity = [pd.NA]
for i in range(1, len(statement_sets)):
    A = statement_sets.iloc[i]
    B = statement_sets.iloc[i-1]
    if not A and not B:
        statement_similarity.append(pd.NA)
    else:
        statement_similarity.append(jaccard(A, B))

df["statement_similarity"] = statement_similarity

df[["date", "statement_similarity"]].head(10)


,date,statement_similarity
0,1999-01-07,<NA>
1,1999-02-04,0.12491
2,1999-03-04,0.157895
3,1999-04-08,0.09764
4,1999-05-06,0.103343
5,1999-06-02,0.147335
6,1999-07-15,0.100071
7,1999-09-09,0.085427
8,1999-10-07,0.106396
9,1999-11-04,0.099407


In [ ]:
# Step 1.4 — Export statement similarity to Drive

import os

BASE_PATH = "/content/drive/MyDrive/PROJET ECB"   # adapte si besoin
out_path = os.path.join(BASE_PATH, "statement_similarity.csv")

df[["date", "statement_similarity"]].to_csv(out_path, index=False)

print("Statement similarity saved to:", out_path)


Statement similarity saved to: /content/drive/MyDrive/PROJET ECB/statement_similarity.csv


Interpretation — Statement Similarity

We compute a measure of textual similarity between consecutive ECB introductory statements using a Jaccard index applied to bigrams. This approach captures the degree to which the ECB repeats or modifies its communication from one press conference to the next. The resulting similarity values range between approximately 0.08 and 0.15 in the early part of the sample, indicating a moderate level of persistence in the wording of the statements. These results suggest that while the ECB maintains a relatively stable communication framework, the content of its statements evolves over time, reflecting changes in economic conditions and policy priorities. This variable therefore provides a quantitative proxy for the stability of central bank communication over time.

In [26]:
# Step 1.5 — Q&A similarity over time

qa_sets = df["qa_text_clean"].apply(bigram_set)

qa_similarity = [pd.NA]
for i in range(1, len(qa_sets)):
    qa_similarity.append(
        jaccard(qa_sets.iloc[i], qa_sets.iloc[i-1])
    )

df["qa_similarity"] = qa_similarity

df[["date", "qa_similarity"]].head(10)


,date,qa_similarity
0,1999-01-07,<NA>
1,1999-02-04,0.034991
2,1999-03-04,0.035145
3,1999-04-08,0.035402
4,1999-05-06,0.033854
5,1999-06-02,0.036081
6,1999-07-15,0.036747
7,1999-09-09,0.03633
8,1999-10-07,0.036701
9,1999-11-04,0.034682


In [ ]:
# Step 1.5 — Export Q&A similarity to Drive

import os

BASE_PATH = "/content/drive/MyDrive/PROJET ECB"
out_path = os.path.join(BASE_PATH, "ecb_step1_5_qa_similarity.csv")

df[["meeting_date", "date", "qa_similarity"]].to_csv(out_path, index=False)

print("Q&A similarity saved to:", out_path)


Q&A similarity saved to: /content/drive/MyDrive/PROJET ECB/ecb_step1_5_qa_similarity.csv


In [ ]:
# Optional — Compare distributions

df[["statement_similarity", "qa_similarity"]].dropna().describe()


,statement_similarity,qa_similarity
count,273.000000,273.0
unique,273.000000,269.0
top,0.290786,0.0
freq,1.000000,4.0


We extend the textual similarity analysis to the question-and-answer (Q&A) sessions of ECB press conferences. Using the same Jaccard similarity measure applied to bigrams, we compare each Q&A session with the one from the previous conference. The resulting similarity values are generally lower and more volatile than those observed for the introductory statements. This suggests that while the ECB maintains a relatively stable and carefully structured communication in its prepared statements, the Q&A sessions are more flexible and responsive to current economic conditions and journalists’ concerns. These findings highlight the complementary nature of the two communication channels.